In [1]:
import sys
import numpy as np
import cv2

In [30]:
# img = cv2.imread('./peoples.jpg')
img = cv2.imread('./kite.jpg')
# img = cv2.imread('./dog.jpg')

if img is None:
    print('image read failed')
    sys.exit()
    
## model import

model = './yolov3.weights'
config = './yolov3.cfg.txt'

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('model load failed')
    sys.exit()
    
## class names
classNames = []
with open('./coco.names.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

# print(output_layers)
# print(layer_names)    
# display(classNames)

blob = cv2.dnn.blobFromImage(img, 1/255., (320,320), swapRB=True)
net.setInput(blob)
output = net.forward(output_layers)

colors = np.random.uniform(0, 255, size = (80, 3))

h, w = img.shape[:2]

class_ids = []
confidences = []
boxes = []

for out in output:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.5:
            
            cx = int(detection[0] * w)
            cy = int(detection[1] * h)
            bw = int(detection[2] * w)
            bh = int(detection[3] * h)
            
            sx = int(cx - bw/2)
            sy = int(cy - bh/2)
            
            boxes.append([sx, sy, bw, bh])
            confidences.append(confidence)
            class_ids.append(int(class_id))
            
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.55)

for i in indices:
    sx, sy, bw, bh = boxes[i]
    label = f'{classNames[class_ids[i]]}, {confidences[i]*100:.2f}%'
    color = colors[class_ids[i]]
    cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
    cv2.putText(img, label, (sx, sy-5), cv2.FONT_HERSHEY_COMPLEX,0.8, color, 1, cv2.LINE_AA)
    
cv2.imshow('image',img)
cv2.waitKey()
cv2.destroyAllWindows()

# display(boxes)
# display(indices)